In [1]:
import os
import csv
import numpy as np
import pandas as pd
import re
import nltk
from sklearn.datasets import load_files 
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
from sklearn.ensemble import RandomForestClassifier
import pickle
import random
import time
from tqdm import tqdm

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\natal_000\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\natal_000\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Options

In [2]:
debug = False

output_path = r"C:\Users\natal\Desktop\Korpus\debug_1812"
#output_path = r'D:\Natalie\13_korpus\run_bote_210216'
output_path = r'D:\Natalie\13_korpus\run_mag_210216'

### 2 Importing training data and write to requested format

In [3]:
Aussagen_neg = r"D:\Natalie\09_ML\Aussagen_neg.txt"
Aussagen_pos = r"D:\Natalie\09_ML\Aussagen_pos.txt"
output_pos = r"D:\Natalie\09_ML\ls_saetze\pos"
output_neg = r"D:\Natalie\09_ML\ls_saetze\neg"

with open(Aussagen_pos, encoding="utf-8") as input_file:
    for i, line in enumerate(input_file.readlines()):
        with open(f"{output_pos}/{i:03d}.txt", 'w', encoding="utf-8") as output_file:
            output_file.write(line)
with open(Aussagen_neg, encoding="utf-8") as input_file:
    for i, line in enumerate(input_file.readlines()):
        with open(f"{output_neg}/{i:03d}.txt", 'w', encoding="utf-8") as output_file:
            output_file.write(line)

In [4]:
landscape_data = load_files(r"D:\Natalie\09_ML\ls_saetze", encoding="utf-8")
X, y = landscape_data.data, landscape_data.target

In [5]:
len(X)

481

In [6]:
def check_words_in_sentence(words, sentence):
    return any(word in sentence for word in words)

In [7]:
def preprocessing(X, lsb=[], ev_lsb=[], kontextwoerter=[], manual=False):
    #assert(not manual or (lsb and ev_lsb and kontextwoerter))
    documents=[]
    debug_info=[]
    list_lsb = []
    #patterns1 = "123456789thiswillhopefullynevergetmatched"
    patterns2 = "123456789thiswillhopefullynevergetmatched"
    #patterns3 = "123456789thiswillhopefullynevergetmatched"
    patterns4 = "123456789thiswillhopefullynevergetmatched"
    #from nltk.stem import WordNetLemmatizer
    #stemmer = WordNetLemmatizer()
    
    
    #1. Wortabwandlungen von definitiven LS-Begriffen
    #for ls in lsb:
     #   patterns1 += f"|{ls}[\\b"
      #  for ending in ["er", "in", "innen", "Innen", "s", "en", "ers", "erisch", "isch", "e", "n", "ig", "es", "ieren", "eln", "ung"]:
       #     patterns1 += f"|{ending}\\b"
        #patterns1 += "]"
        
    #2. Kompositionswörter / Wortzusammensetzung
    for compound in ["ebene", "graben", "gräben", "kanal", "sonne", "gebiet", "wolken", "gebiete", "landschaften", "landschaft", "nebel", "park", "tour", "touren", "wanderung"]:
        patterns2 += f"|[A-Z]\\w*{compound}\\b"
    for compound in ["Älp", "Birken", "Buchen", "Damm", "Dämm", "Eis",  "Hafen", "Nebel", "Wolke", "Rasen"]:
    #["[A|Ä]lp", "Berg", "Birken", "Buchen", "Damm", "Dämm", "Dunst", "Eis", "Fels", "Fluss", "Gebiet", "Hafen", "Hügel", "Nebel", "Wolke", "Rasen", "Schnee", "Schwing", "See", "Sonne", "Stein", "Ufer", "Wander", "Wasser", "Wiesen", "Wild"]:
        patterns2 += f"|\\b{compound}\\w*"
      
    #3. Kontext im Satz: Satz wird rausgespeichert, wenn er einen eventuellen LS-Begriff enthält und zusätzlich mindestens ein Schlüsselwort
    for kontextwort in kontextwoerter+lsb:
        patterns4 += ""#f"|[A-Z][^\\.?!]*({kontextwort})[^\\.?!]*[\\.?!]"#kontextwort in sentence
    
    time_dict = {1: 0, 2: 0, 3: 0, 4: 0}
    for sen in tqdm(X):
        # Remove all the special characters
        document = re.sub(r'\W', ' ', str(sen))
        # remove all single characters
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)  
        # Remove single characters from the start
        document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)  
        # Substituting multiple spaces with single space
        document = re.sub(r'\s+', ' ', document, flags=re.I) 
        # Removing prefixed 'b'
        document = re.sub(r'^b\s+', '', document)  

        sen = document
        
        if manual:
            d_info = {"sentence": sen, "lsb": []}
            tic = time.perf_counter()
            #results = list(re.finditer(patterns1, sen))
            
            for ls in lsb:
                if ls + " " in sen or ls + "," in sen:
                    d_info["lsb"].append({"ls-wort": ls, "type": "lsb"})
                    #if sen not in documents:
                        #documents.append(sen)
                        
            toc = time.perf_counter()
            time_dict[1] += toc-tic
            
            #if results:
             #   documents.append(sen)
              #  debug_info.append((1, [res.string[res.start():res.end()] for res in results]))   
            # continue

            tic = time.perf_counter() 
            results = list(re.finditer(patterns2, sen))
            toc = time.perf_counter()
            time_dict[2] += toc-tic
            if results:
                #documents.append(sen)
                for res in results:
                    d_info["lsb"].append({"ls-wort": res.string[res.start():res.end()], "type": "compound"})

            if check_words_in_sentence(kontextwoerter+lsb, sen):
                for e_lsb in ev_lsb:
                    #patterns3 = f"{e_lsb}\\b"
                    #for ending in ["er", "in", "innen", "Innen", "s", "en", "ers", "erisch", "isch", "e", "n", "ig", "es", "ieren", "eln", "ung"]:
                     #   patterns3 += f"|{e_lsb}{ending}\\b"
                    tic = time.perf_counter()
                    if e_lsb + " " in sen or e_lsb + "," in sen:
                        d_info["lsb"].append({"ls-wort": e_lsb, "type": "ev_lsb"})
                        #if sen not in documents:
                            #documents.append(sen)
                #results = list(re.finditer(patterns3, sen))
                    toc = time.perf_counter()
                    time_dict[3] += toc-tic
                    #if results:
                     #   documents.append(sen)
                      #  debug_info.append((3, [res.string[res.start():res.end()] for res in results]))  
            
            if d_info["lsb"]:
                debug_info.append(d_info)
                documents.append(sen)
            
        else:
            documents.append(sen)
    print(time_dict)
        
    return documents, debug_info

In [8]:
# 4 Converting Text to Numbers
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfconverter = TfidfVectorizer(max_features=2000, min_df=1, max_df=0.8, stop_words=stopwords.words('german'), ngram_range=(1, 3))
    
def to_numbers(X_pre, fit=True):
    #Term frequency = (Number of Occurrences of a word)/(Total words in the document)
    #IDF(word) = Log((Total number of documents)/(Number of documents containing the word))

    if fit:
        X_num = tfidfconverter.fit_transform(X_pre).toarray()
    else:
        X_num = tfidfconverter.transform(X_pre).toarray()
    return X_num

In [9]:
X_pre, _ = preprocessing(X)
X_num = to_numbers(X_pre)
X_num.shape

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 481/481 [00:00<00:00, 30777.89it/s]

{1: 0, 2: 0, 3: 0, 4: 0}


(481, 2000)

In [10]:
# 5 Training and Test Sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_num, y, test_size=0.2, random_state=1)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(384, 2000) (97, 2000) (384,) (97,)


In [11]:
# 6 Training Text Classification Model and Predicting Sentiment
classifier = RandomForestClassifier(n_estimators=150, random_state=12, max_depth=3000)
classifier.fit(X_train, y_train)     

y_pred = classifier.predict(X_test)

In [12]:
# 7 Evaluating The Model
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[55  0]
 [17 25]]
              precision    recall  f1-score   support

           0       0.76      1.00      0.87        55
           1       1.00      0.60      0.75        42

    accuracy                           0.82        97
   macro avg       0.88      0.80      0.81        97
weighted avg       0.87      0.82      0.81        97

0.8247422680412371


In [13]:
# 8 Saving the Model
with open(os.path.join(output_path, 'text_classifier.pkl'), 'wb') as picklefile:
    pickle.dump(classifier,picklefile)

In [14]:
# 9 Loading the Model
with open(os.path.join(output_path, 'text_classifier.pkl'), 'rb') as picklefile:
    classifier = pickle.load(picklefile)

### Loading LSB

In [15]:
# Idee:
# Im Text nach LS-Wörtern suchen.
# Satz mit LS-Wort rausspeichern. // oder umgekehrt 
# Sätze mit LS-Wort klassieren: ls-relevant oder nicht. // oder umgekehrt
# ???
# Profit

# Lade die Liste der LS-Begriffe
ls_begriffe = r"D:\Natalie\05_LS-Begriffe\LS_Begriffe.csv"
with open(ls_begriffe, 'r', encoding='latin1') as infile:
# read the file as a dictionary for each row ({header : value})
    reader = csv.DictReader(infile, delimiter=";")
    data = {}
    for row in reader:
        for header, value in row.items():
            if value:
                if header in data:
                    data[header].append(value)
                else:
                    data[header] = [value]
lsb = data['Definitive Begriffe']
ev_lsb = data['Begriffe im LS-Kontext']
kontextwoerter = data['Kontextwoerter']

In [16]:
## für mehrere Files, die in Sätze gesplittet werden müssen
#no_context = []
#for text_file in tqdm(os.listdir(text_files_pipeline)):
#    with open(os.path.join(text_files_pipeline, text_file), encoding='utf-8') as file:
#        data = '\n'.join(file.readlines())
#        sentences = re.split(r' *[\.\?!][\'"\)\]]* *', data)
#        no_context += sentences

In [17]:
## für ein File, das schon pro Linie einen Satz enthält
#no_context = []
#with open(text_files_pipeline, encoding='utf-8') as file:
#    no_context = file.readlines()

In [18]:
## für Sätze aus Picklefile
#no_context = []
#for sen in files_frompickle:
#    print(sen)
#    no_context.append(sen[0])

# HERE

In [19]:
#tic = time.perf_counter()
#eval_pre, debug_info = preprocessing(X=no_context, lsb=lsb, ev_lsb=ev_lsb, kontextwoerter=kontextwoerter, manual=True)
#toc = time.perf_counter()
#print(f"{(toc-tic)*1000:.2f}ms ({(toc-tic)/0.63*100:.2f}%)")
#print(len(eval_pre))
#eval_num = to_numbers(eval_pre, fit=False)
#eval_num.shape

#eval_pred = classifier.predict(eval_num)

In [20]:
for temp_topo_file in os.listdir(output_path):
    if temp_topo_file.startswith("temp_topo"):
        with open(os.path.join(output_path, temp_topo_file), 'rb') as fileo:
            print(os.path.join(output_path, temp_topo_file))
            data_frompickle = pickle.load(fileo)

    
        tic = time.perf_counter()
        eval_pre, debug_info = preprocessing(X=data_frompickle["saetze"], lsb=lsb, ev_lsb=ev_lsb, kontextwoerter=kontextwoerter, manual=True)
        print("perfom preprocessing: ", temp_topo_file) 
        assert(len(eval_pre)==len(debug_info))
        toc = time.perf_counter()
        print(f"{(toc-tic)*1000:.2f}ms ({(toc-tic)/0.63*100:.2f}%)")
        eval_num = to_numbers(eval_pre, fit=False)
        print(eval_num.shape)
        print("perfom classifier: ", temp_topo_file) 
        eval_pred = classifier.predict(eval_num)
        print(eval_pred)
        #save all 10 iterations    
        pickle_data = {
            "eval_pre":eval_pre , 
            "debug_info":debug_info, 
            "eval_num":eval_num, 
            "eval_pred":eval_pred, 
        }
        with open(f"{output_path}/temp_lsb_{temp_topo_file.split('_')[-1]}", 'wb') as file:
            pickle.dump(pickle_data, file)
    
    else:
        pass
        #print(temp_topo_file)
    

  0%|                                                                                                                                                            | 0/3635 [00:00<?, ?it/s]

D:\Natalie\13_korpus\run_mag_210216\temp_topo_0.pkl


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3635/3635 [00:03<00:00, 941.75it/s]


{1: 2.3529416809994927, 2: 0.3340428820007446, 3: 0.21506887700110155, 4: 0}
perfom preprocessing:  temp_topo_0.pkl
3859.26ms (612.58%)
(911, 2000)
perfom classifier:  temp_topo_0.pkl
[0 0 1 1 0 1 1 0 1 1 1 0 0 0 1 1 1 1 1 0 1 1 0 0 1 1 1 1 0 0 1 0 1 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 1 0 1 0 0 1 0
 0 0 1 0 0 0 0 1 0 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1
 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 1 1 1 1
 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 1 0 0 1 0 0 1 0 0 0 1 1 0 0 0 0 0
 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 1 1 0 1
 0 0 0 0 1 0 1 1 1 0 0 0 0 1 1 0 1 0 1 1 0 1 1 1 1 0 0 0 0 0 0 0 0 0 1 0 0
 1 0 1 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 1 0
 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0
 1 1 0 0 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 0 1 0 1 0 0

In [21]:
for debinf in debug_info:
    print("debinfo:", debinf, "\n")
for i in range(0,len(debug_info)):
    print(i)
    print("di:", debug_info[i])

debinfo: {'sentence': 'Y Nº 01 02 ZIMMER MIT AUSSICHT Blick auf Schwyz vom Feldli aus 03 EIGH BOU HOOD 08 Die Mitte der Schweiz liegt mitten in Schwyz', 'lsb': [{'ls-wort': 'Blick', 'type': 'ev_lsb'}]} 

debinfo: {'sentence': 'com GR K Florian Fischer TA BE R DI SE AUSGA BE Nathalie Henseler Hanjo Seißler Hans Steinegger Urs Derendinger Illustrationen Burkhard Neie Fotos Stefan Zürrer Heinz Dahinden Manuela Jans Filipa Peixeiro NSC F DE DA ION MAG Hauptplatz 10 6430 Schwyz DRUC Gutenberg Druck AG Lachen WA RU DI SE AGA Y S N WA W M WOR Verschlaucht So ist die Immenseer Künstlerin Verena Vanoli auf den Schlauch gekommen', 'lsb': [{'ls-wort': 'Fischer', 'type': 'lsb'}]} 

debinfo: {'sentence': 'Schließlich haben wir bis jetzt zwar zähneknirschend aber anerkennend genickt wenn nach dieser Frage immer gleich die im Kanton Uri liegende Bergwiese Rütli zur Hand war', 'lsb': [{'ls-wort': 'Bergwiese', 'type': 'lsb'}]} 

debinfo: {'sentence': 'In diesem Jahrhundert als zum ersten Mal Vor und Na


debinfo: {'sentence': 'Und das alte glückliche Geleise weist den Weg in die Zukunft So lange der Schwyzer die Ehre in der Tugend sucht wird sie ihm auf rechte Pfade leuchten', 'lsb': [{'ls-wort': 'Weg', 'type': 'ev_lsb'}]} 

debinfo: {'sentence': 'Böse Zungen könnten nun behaupten Amerikaner dächten ohnehin die Butter komme aus dem Kühlregal und kämen im Alpsummer ins Staunen weil sie erstmals sähen wie vier Generationen auf Schweizer Alpen leben melken buttern und käsen', 'lsb': [{'ls-wort': 'Alpen', 'type': 'lsb'}]} 

debinfo: {'sentence': ' Deshalb bin ich ins bündnerische Calanda Tal wo er häufiger anzutreffen ist als bei uns und habe von Bauern und Experten erfahren wie der Wolf tatsächlich ist Er ist ein Opportunist der den einfachsten Weg sucht zu seiner Fleischration zu kommen', 'lsb': [{'ls-wort': 'Tal', 'type': 'lsb'}, {'ls-wort': 'Weg', 'type': 'ev_lsb'}]} 

debinfo: {'sentence': '18 18 25 Rot leuchtet der Turm auf der Rigi weithin ins Land FOTO Stefan Zürrer Y26_2018_08_04

di: {'sentence': 'Stephan Rothlin Hans Peter Häﬂ iger Guido Henseler Eugen Haug Erwin Horat Benno Kälin Franz Xaver Risi sowie Nik Oswald SCH LUS SR DA ION Hanjo Seißler FOTOS Stefan Zürrer James Ewing Ugo Rondinone und Lucia Morate Ugo Rondinone LUS AT ION Michael Pleesz Porträts und Florian Fischer Collagen HO Sophia Plazotta PX5 NSC F DE DA ION MAG Feldli 6430 Schwyz DRUC Gutenberg Druck AG Lachen Y12_Magazin_fin_47L_BZG', 'lsb': [{'ls-wort': 'Fischer', 'type': 'lsb'}]}
80
di: {'sentence': 'Der Tobelbach rauscht tief unten talwärts ehe sich nach der Mittelstation eine kühne Seilkonstruktion auf die Rotenﬂue emporschwingt', 'lsb': [{'ls-wort': 'Mittelstation', 'type': 'lsb'}]}
81
di: {'sentence': ' Den Grossen Mythen im Rücken schaut man staunend westwärts vom sagenumwobenen Vrenelisgärtli in Glarus über Bös Fulen dem mit 2802 Metern höchsten Schwyzer Berg und die Druesbergkette bis zum Kaiserstock und Klingenstock ', 'lsb': [{'ls-wort': 'Berg', 'type': 'lsb'}]}
82
di: {'sentence': '

654
di: {'sentence': ' ILLUSTRATION Anisonk Thongra Ar von Andreas Lukoschik eim Betrachten der Stationen im Leben der Gabriella Baumann von Arx ist unübersehbar dass hier ein Leben im Fluss ist Arztgehilfin Swissair Flugbegleiterin Journalistin Bestseller Autorin und seit 15 Jahren Chefin eines Verlages der zu den erfolgreichen im Lande gehört', 'lsb': [{'ls-wort': 'Fluss', 'type': 'lsb'}]}
655
di: {'sentence': ' In Erinnerung an diesen Himmelsboten hängt übrigens ein grosses Foto eines Spatzen im Eingangsbereich ihres Verlagsbüros in Lachens Herrengasse Nr 3', 'lsb': [{'ls-wort': 'Foto', 'type': 'ev_lsb'}]}
656
di: {'sentence': 'indd 43 12 Wochen in den Charts Mutanfall Mein Leben ohne Ernst 11 Wochen in den Charts für immer Die unfassbare Tat von Rupperswil und ihre Folgen 11 Wochen in den Charts Der Blindgänger Das gewagte Leben des Steven Mack 11 Wochen in den Charts Lotti Latrous Bangen und Hoffen im Slum von Abidjan 10 Wochen in den Charts Die Bestseller der Schwyzer Autorin Bla

In [22]:
eval_pre=[]
debug_info=[]
eval_num=[]
eval_pred=[]

for temp_lsb_file in os.listdir(output_path):
    if temp_lsb_file.startswith("temp_lsb"):
        with open(os.path.join(output_path, temp_lsb_file), 'rb') as fileo:
            #print(os.path.join(data_files_frompickle, file))
            data = pickle.load(fileo)
            eval_pre += data["eval_pre"]
            debug_info += data["debug_info"]
            eval_num.append(data["eval_num"])
            eval_pred.append(data["eval_pred"])

eval_num = np.concatenate(eval_num)
eval_pred = np.concatenate(eval_pred)

In [23]:
print(sum(eval_pred), len(eval_pred))
print(eval_num.shape)
print(len(debug_info))
print(len(eval_pre))

248 911
(911, 2000)
911
911


In [24]:
for eval_prediction, debinf in zip(eval_pred, debug_info_10):
    if eval_prediction == 1:
        print(eval_prediction, ":--------")
        print(debinf, "\n")
for i in range(0,len(debug_info_10)):
    print(i)
    print(debug_info[i][1])

NameError: name 'debug_info_10' is not defined

In [70]:
landschafts_begriffe=[]
for res, sen, debinf in zip(eval_pred, eval_pre, debug_info):
    if res==1:
        for i in range(0,len(debinf)):
            print(len(debinf))
            print(i)
            print(debinf[i][1])
            landschafts_begriffe.append(debinf[i][1])
        print("-", sen, "\n")

2
0


KeyError: 0

In [34]:
len(landschafts_begriffe)

0

In [59]:
many_lsb_in_sen=0
for res, sen, debinf in sorted(zip(eval_pred, eval_pre, debug_info)):
    if res==1 and len(debinf[0])>1:
        many_lsb_in_sen+=1
        print("-------------------------")
        print(debinf, "-", sen)

TypeError: '<' not supported between instances of 'dict' and 'dict'

In [70]:
many_lsb_in_sen

197

In [94]:
output = r"C:\Users\natal\Desktop\Studium\01 Masterarbeit\08_Evaluation\topoUNDlsb"

with open(f"{output}/lsb_topo_1-10.csv", 'w', encoding="utf-8") as output_file:
    for res, sen in sorted(zip(eval_pred, eval_pre)):
        if res == 1:
            output_file.write(sen+"\n")

In [275]:
results_neg=[]
results_pos=[]
results = zip(eval_pred, eval_pre)

for result, context in zip(eval_pred, eval_pre):
    if result == 0:
        results_neg.append(context)
    else:
        results_pos.append(context)

assert(len(results_pos) == sum(eval_pred))
print(len(results_pos))
print(len(results_neg))
print(len(eval_pre))

random.seed(1)
random_pos = random.sample(results_pos, k=60)
#12 yay, 48 nay (precision: 0.25)
#10 yay ,50 nay (precision: 0.2)
#13, 47
#13, 47
#optimised: 36 ,24
#optimised, larger: 42,25
#mehr als neg klassierte inputsätze: 39, 21
random.seed(15)
random_neg = random.sample(results_neg, k=40)
#34 yay (also korrekt klassiert), 6 nay
#37 yay (also korrekt klassiert), 3 nay
#36, 4
#36, 4
#optimised: 29, 11
#optimised, larger: 31, 9
#mehr als neg klassierte inputsätze: 35, 5
random_ohneML = random.sample(eval_pre, k=100)

1246
14204
15450


In [277]:
#Precision
#True Positives / True Positives + False Positives
print("precision round 1: ",36/(36+24))
print("precision round 2: ",42/(42+25))
print("precision round 3: ",39/(39+21))
print("precision round random auswahl 1: ",34/(34+66))
print("precision round random auswahl 2: ",29/(29+71), "\n")

#Recall
#True Positives / True Positives + False Negatives
print("recall round 1: ",36/(36+11))
print("recall round 2: ",42/(42+9))
print("recall round 3: ",39/(39+5))

precision round 1:  0.6
precision round 2:  0.6268656716417911
precision round 3:  0.65
precision round random auswahl 1:  0.34
precision round random auswahl 2:  0.29 

recall round 1:  0.7659574468085106
recall round 2:  0.8235294117647058
recall round 3:  0.8863636363636364


In [95]:
print(len(no_context))
print(len(res))

6439


TypeError: object of type 'numpy.int32' has no len()

In [276]:
output = r"C:\Users\natal\Desktop\Studium\01 Masterarbeit\08_Evaluation\eval_ML"

#with open(f"{output}\pos_klassiert.txt", 'w', encoding="utf-8") as output_file:
#    for sentence in results_pos:
#        output_file.write("-------------------------%s\n" % sentence)
with open(f"{output}\\random_ohneklass.txt", 'w', encoding="utf-8") as output_file:
    for sentence in random_ohneML:
        output_file.write("-------------------------%s\n" % sentence)

In [185]:
# 10 Evaluation
# Idee:
# Im Text nach LS-Wörtern suchen.
# Satz mit LS-Wort rausspeichern. // oder umgekehrt  <<<--------
# Sätze mit LS-Wort klassieren: ls-relevant oder nicht. // oder umgekehrt
# ???
# Profit

# 9 Applying the Model
with open('text_classifier', 'rb') as training_model:
    model = pickle.load(training_model)

print(confusion_matrix(y_test, y_pred2))
print(classification_report(y_test, y_pred2))
print(accuracy_score(y_test, y_pred2)) 

[[33  2]
 [ 9 18]]
              precision    recall  f1-score   support

           0       0.79      0.94      0.86        35
           1       0.90      0.67      0.77        27

    accuracy                           0.82        62
   macro avg       0.84      0.80      0.81        62
weighted avg       0.84      0.82      0.82        62

0.8225806451612904


In [35]:
text = "hallo, wie geht's?"
pattern = "hallo"
result = re.finditer(pattern, text)
for res in result:
    print("hi")
    print(type(res))
    print(res)
print(result)
print(len(list(result)))


hi
<class 're.Match'>
<re.Match object; span=(0, 5), match='hallo'>
0


### Test nach fehlenden Endungen

In [ ]:
# Preprocessing und jetzt nach allen LSB etc. suchen, die nicht diese Endungen haben.
def test_preprocessing(X, lsb=[], ev_lsb=[], kontextwoerter=[], manual=False):
    assert(not manual or (lsb and ev_lsb and kontextwoerter))
    endungen=[]
    documents=[]
    debug_info=[]
    list_lsb = []
    patterns1 = "123456789thiswillhopefullynevergetmatched"
    patterns2 = "123456789thiswillhopefullynevergetmatched"
    patterns3 = "123456789thiswillhopefullynevergetmatched"
    patterns4 = "123456789thiswillhopefullynevergetmatched"
    #from nltk.stem import WordNetLemmatizer
    #stemmer = WordNetLemmatizer()
    
    #1. Wortabwandlungen von definitiven LS-Begriffen
    for ls in lsb:
        #patterns1 += f"|{ls}\\b"
        patterns1 += f"|{ls}[\\b"
        for ending in ["er", "in", "innen", "Innen", "s", "en", "ers", "erisch", "isch", "e", "n", "ig", "es", "ieren", "eln", "ung"]:
            patterns1 += f"|{ending}\\b"
        patterns1 += "]"
        
    #2. Kompositionswörter / Wortzusammensetzung
    for compound in ["ebene", "graben", "gräben", "kanal", "sonne", "weg", "gebiet", "wolken", "gebiete", "landschaften", "landschaft", "nebel", "park", "wanderung"]:
        patterns2 += f"|[A-Z]\\w*{compound}\\b"
    for compound in ["Älp", "Birken", "Buchen", "Damm", "Dämm", "Eis",  "Hafen", "Nebel", "Wolke", "Rasen"]:
    #["[A|Ä]lp", "Berg", "Birken", "Buchen", "Damm", "Dämm", "Dunst", "Eis", "Fels", "Fluss", "Gebiet", "Hafen", "Hügel", "Nebel", "Wolke", "Rasen", "Schnee", "Schwing", "See", "Sonne", "Stein", "Ufer", "Wander", "Wasser", "Wiesen", "Wild"]:
        patterns2 += f"|\\b{compound}\\w*"
      
    #3. Kontext im Satz: Satz wird rausgespeichert, wenn er einen eventuellen LS-Begriff enthält und zusätzlich mindestens ein Schlüsselwort
    for kontextwort in kontextwoerter+lsb:
        patterns4 += ""#f"|[A-Z][^\\.?!]*({kontextwort})[^\\.?!]*[\\.?!]"#kontextwort in sentence
    
    time_dict = {1: 0, 2: 0, 3: 0, 4: 0}
    for sen in tqdm(X):
        # Remove all the special characters
        document = re.sub(r'\W', ' ', str(sen))
        # remove all single characters
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)  
        # Remove single characters from the start
        document = re.sub(r'\^[a-zA-Z]\s+', ' ', document)  
        # Substituting multiple spaces with single space
        document = re.sub(r'\s+', ' ', document, flags=re.I) 
        # Removing prefixed 'b'
        document = re.sub(r'^b\s+', '', document) 
        # Converting to Lowercase
        #document = document.lower()  
        # Lemmatization
        #document = document.split()
        #document = [stemmer.lemmatize(word) for word in document]
        #document = ' '.join(document)
        #documents.append(document)
        #debug_info.append((4, []))
        sen = document
        
        if manual:
            tic = time.perf_counter()
            for word in sen.split(" "):
                for ls in lsb:
                    if ls in word and word not in patterns1:
                        if word not in endungen:                        
                            endungen.append(word)
                
            toc = time.perf_counter()
            time_dict[1] += toc-tic

            if check_words_in_sentence(kontextwoerter+lsb, sen):
                for e_lsb in ev_lsb:
                    patterns3 = f"{e_lsb}\\b"
                    for ending in ["er", "in", "innen", "Innen", "s", "en", "ers", "erisch", "isch", "e", "n", "ig", "es", "ieren", "eln", "ung"]:
                        patterns3 += f"|{e_lsb}{ending}\\b"
                    tic = time.perf_counter()
                for word in sen.split(" "):
                    for e_lsb in ev_lsb:
                        if e_lsb in word and word not in patterns3:
                            if word not in endungen:                        
                                endungen.append(word)
                    
                    toc = time.perf_counter()
                    time_dict[3] += toc-tic 
                    break

        else:
            documents.append(sen)
    print(time_dict)
        
    return endungen, debug_info

In [ ]:
endungen_test, debug_info = test_preprocessing(X=no_context, lsb=lsb, ev_lsb=ev_lsb, kontextwoerter=kontextwoerter, manual=True)
print(sorted(endungen_test)) #Grat

In [ ]:
doc_ex, debug_info_ex = preprocessing(["auf der Alp bewundern wir die Kühe, die im Grasgraben stehen"], lsb=["Alp"], ev_lsb=["Kühe"], kontextwoerter=["bewundern"], manual=True)
debug_info_ex

In [ ]:
# from germanetpy import germanet
# germanet_object = germanet.Germanet(r"X:\nbranden\data\Documents\GN_V150\GN_V150\GN_V150_XML")

# load 4 parsers with good results in the CoNLL 2018 Shared Task
# stanfordnlp, neuerdings genannt stanza
#import stanza
#stanza.download("de")
# turku

# COMBO

# UDPipe Future 

#from german_lemmatizer import lemmatize

#res=lemmatize(
 #   ['Johannes war ein guter Schüler', 'Sabiene sang zahlreiche Lieder'],
  #  working_dir='*',
   # chunk_size=10000,
    #n_jobs=1,
    #escape=False,
    #remove_stop=False)

# function: do the preprocessing with 4 different lemmatizers

        #Lemmatization
        #document = document.split()
        #document = [stemmer.lemmatize(word) for word in document]
        #document = ' '.join(document)
        #documents.append(document)
        #debug_info.append((4, []))

#nlp = stanza.Pipeline('de') # This sets up a default neural pipeline in English
#doc = nlp("Wir sind gewandert und es war schön wegen der vielen Extensievierungen. Weniger schön waren die vielen Gewächshäuser und die Uferbebauungen.")
#doc.sentences[0].print_dependencies()
#print(*[f'word: {word.text+" "}\tlemma: {word.lemma}' for sent in doc.sentences for word in sent.words], sep='\n')